# tensorflow模型持久化

In [2]:
###################################################################################################
# 持久化模型实例
import tensorflow as tf

# 声明两个变量并计算它们的和
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")

result = v1 + v2

init_op = tf.initialize_all_variables()
# 声明tf.train.Saver类用于保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    saver.save(sess, "/Users/xxx/work5/tensorflow/tensormode/model.cpkt")
    print sess.run(result)


[ 3.]


In [17]:
###################################################################################################
# 加载持久化模型方式一： 
import tensorflow as tf

# # 声明两个变量并计算它们的和
v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="other-v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="other-v2")
result = v1 + v2

saver = tf.train.Saver({"v1": v1, "v2": v2})

with tf.Session() as sess: 
    saver.restore(sess, "/Users/xxx/work5/tensorflow/tensormode/model.cpkt")
    print sess.run(result)


[ 3.]


In [10]:
###################################################################################################
# 加载持久化模型方式二： 
import tensorflow as tf

saver = tf.train.import_meta_graph("/Users/xxx/work5/tensorflow/tensormode/model.cpkt.meta")

with tf.Session() as sess:
    saver.restore(sess, "/Users/xxx/work5/tensorflow/tensormode/model.cpkt")
    print sess.run(tf.get_default_graph().get_tensor_by_name("add:0"))

[ 3.]


# 持久化---滑动平均模型

In [1]:
###################################################################################################
# 
import tensorflow as tf

v = tf.Variable(0, dtype=tf.float32, name="v")

# 在没有声明滑动平均模型时只有一个变量v， 所以下面的语句只会输出“v:0”
for variables in tf.all_variables():
    print variables.name
    
ema = tf.train.ExponentialMovingAverage(0.99)
maintain_average_op = ema.apply(tf.all_variables())

# 在声明滑动平均模型之后，TensorFlow会自动生成一个影子变量

for variables in tf.all_variables():
    print variables.name
    
saver = tf.train.Saver()

with tf.Session() as sess:
    init_op = tf.initialize_all_variables()
    sess.run(init_op)
    sess.run(tf.assign(v, 10))
    sess.run(maintain_average_op)
    # 保存时，tensorflow会将v:0和v/ExponentialMovingAverage:0两个变量都保存下来
    saver.save(sess, "/Users/xxx/work5/tensorflow/tensormode/model_1.cpkt")
    


v:0
v:0
v/ExponentialMovingAverage:0


In [4]:
###################################################################################################
# 直接读取上面的变量的滑动平均值
v = tf.Variable(0, dtype=tf.float32, name="v")
saver = tf.train.Saver({"v/ExponentialMovingAverage": v})

with tf.Session() as sess:
    saver.restore(sess, "/Users/xxx/work5/tensorflow/tensormode/model_1.cpkt")
    print sess.run(v)    # 输出的值，即为原来模型中变量v的滑动平均值

0.0999999


In [1]:
###################################################################################################
# variables_to_restore函数来生成tf.Train.Saver类所需要的变量重命名字典

import tensorflow as tf

v = tf.Variable(0, dtype=tf.float32, name="v")
ema = tf.train.ExponentialMovingAverage(0.99)
print ema.variables_to_restore()

saver = tf.train.Saver(ema.variables_to_restore())
with tf.Session() as sess:
    saver.restore(sess, "/Users/xxx/work5/tensorflow/tensormode/model_1.cpkt")
    print sess.run(v)


{u'v/ExponentialMovingAverage': <tensorflow.python.ops.variables.Variable object at 0x103158a50>}
0.0999999


In [10]:
###################################################################################################
# tensorflow 通过convert_variables_to_constants将计算图中的变量及其取值通过常量的方式保存
import tensorflow as tf
from tensorflow.python.client import graph_util

v1 = tf.Variable(tf.constant(1.0, shape=[1]), name="v1")
v2 = tf.Variable(tf.constant(2.0, shape=[1]), name="v2")
result = v1 + v2

init_op = tf.initialize_all_variables()
with tf.Session() as sess:
    sess.run(init_op)
    # 导出计算图的GraphDef部分，只需要这一部分就可以完成从输入层到输出层的计算过程
    graph_def = tf.get_default_graph().as_graph_def()
    
    # 将图中的变量以及取值转化为常量，同时将图中不必要的节点去掉
    # 参数["add“]给出了需要保存的节点名称。add节点时上面定义两个变量相加的操作
    output_graph_def = graph_util.convert_variables_to_constants(sess, graph_def, ['add'])
    
    # 将导出的模型存入文件
    with tf.gfile.GFile("/Users/xxx/work5/tensorflow/tensormode/combined_model.pb", "wb") as f:
        f.write(output_graph_def.SerializeToString())



Converted 2 variables to const ops.


In [3]:
###################################################################################################
# 解析通过convert_variables_to_constants保存的模型
import tensorflow as tf
from tensorflow.python.platform import gfile

with tf.Session() as sess:
    model_filename = "/Users/xxx/work5/tensorflow/tensormode/combined_model.pb"
    # 读取保存的模型，并将文件解析成对应的GraphDef Protocol Buffer
    with gfile.FastGFile(model_filename, 'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    
    # 将graph_def中国年的保存的图加载到当前的图中
    # return_elements=["add:0"]给出了返回张量的名称
    # 保存的时候给出的计算节点的名称，所以为“add”， 在加载的时候给出的是张量add:0
    result = tf.import_graph_def(graph_def, return_elements=["add:0"])
    print sess.run(result)
    
    

[array([ 3.], dtype=float32)]
